In [ ]:
%cd ~/src/Sunrise-Demo-Dev/
from __future__ import annotations
try:
    from mediocreatbest import auto, run
except ImportError:
    %pip install --quiet --upgrade pip
    %pip install --upgrade --force-reinstall \
        mediocreatbest@git+https://gist.github.com/player1537/3457b026ed6ef6696d758517f55a58df.git
from mediocreatbest import auto, run

%pip install gpxpy
%pip install scikit-learn
import gpxpy
import gpxpy.gpx

In [ ]:
@run(once=True)
def cache():
    path = auto.pathlib.Path.cwd() / 'tmp' / 'elevation.shelve'
    return auto.shelve.open(str(path), 'c')

In [ ]:
@run
def fetch(cache, /):
    def fetch(url: str, *, tqdm=None) -> bytes:
        key = url
        if key not in cache:
            if tqdm is not None:
                tqdm.set_description(f'Cache Miss: {url}')

            with auto.requests.get(url) as r:
                r.raise_for_status()
                cache[key] = r.content
            
            auto.time.sleep(1)
        
        else:
            if tqdm is not None:
                tqdm.set_description(f'Cache Hit: {url}')
        
        return cache[key]

    return fetch

In [ ]:
import requests
import json
import math
import sklearn
from sklearn.metrics.pairwise import haversine_distances

session = requests.Session()

class Coordinate:
    lat: int
    lng: int
    alt: int

def to_rad(theta):
    return (theta * math.pi) / 180.0

def dist(p1, p2):
    if p1 == None or p2 == None:
        return 10
    
    lat1 = to_rad(p1.latitude)
    lat2 = to_rad(p2.latitude)
    lng1 = to_rad(p1.longitude)
    lng2 = to_rad(p2.longitude)
    d = haversine_distances([[lat1, lng1], [lat2, lng2]])
    return d[0][1] * 6371000

@run
def getCoords(fetch, /):
    with open('trail.gpx') as f:
        gpx = gpxpy.parse(f)
        print(gpx)

        coords = []

        for waypoint in gpx.waypoints:
            print(f'P: {waypoint.latitude}, {waypoint.longitude}')

        for track in gpx.tracks:
            break

        print(f'Number of Segments: {len(track.segments)}')
        for segment in track.segments:
            break

        print(f'Number of Points: {len(segment.points)}')
        prev = None
        good_points = []
        emit = good_points.append

        for point in segment.points:
            lat = point.latitude
            lng = point.longitude
        
            if prev is None or (dist(prev, point) > 1000):
                # print(f'Dist: {dist(prev, point)}')
                prev = point
                emit(point)

        for point in auto.tqdm.auto.tqdm(good_points):
            lat = point.latitude
            lng = point.longitude
            
            url = f'https://api.open-meteo.com/v1/elevation?latitude={lat}&longitude={lng}'
            alt = auto.json.loads(fetch(url))['elevation'][0]

            coords.append(
                {
                    "lat": lat,
                    "lng": lng,
                    "alt": alt
                }
            )
            print(f'{lat}, {lng}, {alt}')

            
            with open('trail.json', 'w') as out:
                json.dump(coords, out, indent=4)
                